In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(rc={'figure.figsize':(12,5)})
%load_ext autoreload
%autoreload 2

In [50]:
import re
from fuzzywuzzy import process
import pprint

In [2]:
df = pd.read_parquet("proceedings.parquet")

In [3]:
df["volume"] = df["conference"].apply(lambda x: x.split(":")[0].split(" ")[1])

In [5]:
df['year'] = df['conference'].apply(lambda x: re.search(r'(19|20)\d{2}', str(x)).group(0) if re.search(r'(19|20)\d{2}', str(x)) else None)

In [6]:
df['year'].value_counts()

year
2022    3229
2023    3106
2021    2834
2020    2407
2019    1840
2018    1255
2017     948
2016     708
2014     574
2015     545
2013     438
2012     284
2011     221
2010     201
2009     111
2007      86
2008      41
2006       5
Name: count, dtype: int64

In [7]:
df["conference_main"] = df["conference"].apply(lambda x: x.split(":")[1].split(',')[0])

In [31]:
df.loc[df['year'].isna(), 'conference'].unique()

array(['Volume 117: Algorithmic Learning Theory, , San Diego, California, USA',
       'Volume 201: International Conference on Algorithmic Learning Theory, , Singapore',
       'Volume 209: Conference on Health, Inference, and Learning, , 415 Main Street, Cambridge, MA USA 02142'],
      dtype=object)

In [34]:
df.loc[df['volume'] == '117', 'year'] = '2020'

In [35]:
df.loc[df['volume'] == '201', 'year'] = '2023'

In [36]:
df.loc[df['volume'] == '209', 'year'] = '2023'

In [44]:
def get_best_match(name, choices):
    return process.extractOne(name, choices)

In [58]:
df['conference_main'] = df['conference_main'].apply(lambda c: c[1:])

In [95]:
# df['conference_main'].apply(remove_prefix)

In [83]:
import networkx as nx

In [120]:
unique_conferences = set(df['conference_main'])

name_mapping = {}
names_graph = nx.Graph()
names_graph.add_nodes_from(unique_conferences)

for conference in unique_conferences:
    match, score = get_best_match(conference, unique_conferences - {conference})
    if score >= 90:
        names_graph.add_edge(conference, match)
        # name_mapping[conference] = match

In [121]:
for c in nx.connected_components(names_graph):
    if len(c) > 1:
        target_name = min(c, key=len)
        for conference in c:
            name_mapping[conference] = target_name

In [122]:
for c1, c2 in name_mapping.items():
    if 'learning theory' in c1.lower():
        print(f'{c1}\n -> {c2}\n')

Algorithmic Learning Theory
 -> Algorithmic Learning Theory

International Conference on Algorithmic Learning Theory
 -> Algorithmic Learning Theory



In [100]:
# def remove_prefix(name):
#     pattern = r'.*?(\b(?!(with|health)\b)\w*th)\b.*?\s'
#     name = re.sub(pattern, '', name).strip()

#     return name

In [99]:
def clean_conference_name(name):
    pattern_prefix = r'.*?(\b(?!(with|health)\b)\w*th)\b.*?\s'
    name = re.sub(pattern_prefix, '', name).strip()
    name = re.sub(r'[0-9\']', '', name)

    return name

In [123]:
df['conference_main'] = df['conference_main'].map(name_mapping).fillna(df['conference_main']).apply(clean_conference_name)

In [124]:
[c for c in df['conference_main'].unique() if 'learning theory' in c.lower()]

['Conference On Learning Theory', 'Algorithmic Learning Theory']

In [132]:
(df[['conference_main', 'volume']].drop_duplicates()['conference_main'].value_counts() > 1).value_counts(normalize=True)

count
False    0.609756
True     0.390244
Name: proportion, dtype: float64

In [127]:
df.loc[df['year'].apply(int) > 2018, 'conference_main'].value_counts(normalize=False).tail(10)

conference_main
Transfer Learning for Natural Language Processing Workshop                                                                              9
European Conference on Machine Learning and Principles and Practice of Knowledge Discovery in Databases                                 7
Proceedings on "I Cant Believe Its Not Better!  - Understanding Deep Learning Through Empirical Falsification" at NeurIPS  Workshops    7
nd Workshop on Online Recommender Systems and User Modeling                                                                             7
Causal Analysis Workshop Series                                                                                                         6
Artificial Intelligence Diversity                                                                                                       5
Workshop on Artificial Intelligence in Affective Computing                                                                              5
Algorithmic Fairne

In [126]:
df[['title', 'authors', 'conference_main', 'year', 'volume']]

,title,authors,conference_main,year,volume
0,Gaussian Process Approximations of Stochastic ...,"Cedric Archambeau, Dan Cornford, Manfred Opper...",Gaussian Processes in Practice,2006,1
1,Multi-class Semi-supervised Learning with the ...,"Simon Rogers, Mark Girolami",Gaussian Processes in Practice,2006,1
2,Learning RoboCup-Keepaway with Kernels,"Tobias Jung, Daniel Polani",Gaussian Processes in Practice,2006,1
3,Salient Point and Scale Detection by Minimum L...,"Kim S. Pedersen, Marco Loog, Pieter Dorst",Gaussian Processes in Practice,2006,1
4,Sparse Log Gaussian Processes via MCMC for Spa...,"Jarno Vanhatalo, Aki Vehtari",Gaussian Processes in Practice,2006,1
...,...,...,...,...,...
18950,Fisher-Rao and pullback Hilbert cone distances...,Frank Nielsen,Topological,2023,221
18951,On Explicit Curvature Regularization in Deep G...,"Yonghyeon Lee, Frank C. Park",Topological,2023,221
18952,MASIL: Towards Maximum Separable Class Represe...,Anant Khandelwal,Topological,2023,221
18953,Topological Feature Selection,"Antonio Briola, Tomaso Aste",Topological,2023,221
